The `seticore::Dedopplerer` class is instantiated with statements of the beamformed data that it will be searching through:

- num_timesteps: the number of spectra in the chunk of data
- num_channels: the number of channels in each spectrum in the chunk of data
- foff: the frequency-width of each channel (MHz)
- tsamp: the timespan of each spectrum (s)

! Note that the Dedopplerer only computes on power-of-2 spectra at a time. !

In [1]:
num_timesteps = 64 # VLASS-spec produces 64 fine-spectra per 8.388 s raw part-files
num_channels = 1*131072 # VLASS-spec processes 1 coarse-channel
foff_hz = 1e6/131072 # COSMIC records 1 MHz channels, upchannelised by a factor of 131072
tsamp_s = 1/foff_hz

import math
assert 2**(math.log2(num_timesteps)) == num_timesteps

print(f"foff_hz: {foff_hz}")
print(f"tsamp_s: {tsamp_s}")

foff_hz: 7.62939453125
tsamp_s: 0.131072


The `Dedopplerer` ultimately ([1](https://github.com/lacker/seticore/blob/1a4275b9a37898e3146da782856deea3a6adcacd/dedoppler.cu#L84-L87) [2](https://github.com/lacker/seticore/blob/1a4275b9a37898e3146da782856deea3a6adcacd/dedoppler.cu#L195)) defines:

$ diagonal\_drift\_rate = \frac{(num\_timesteps-1) \times foff\_hz}{(num\_timesteps-1) \times tsamp\_s} =  \frac{foff\_hz}{tsamp\_s}$

The `max_drift` argument for the `search()` method, is then 'normalised' against `diagonal_drift_rate` ([3](https://github.com/lacker/seticore/blob/1a4275b9a37898e3146da782856deea3a6adcacd/dedoppler.cu#L196)):

$ normalized\_max\_drift = \frac{max\_drift}{|diagonal\_drift\_rate|} $

This `normalized_max_drift` value is the number of `drift_blocks` ('minimal chunk') that will be searched.


Effectively though, the maximum drift-rate that could be found is a natural multiple of `diagonal_drift_rate`.

In [2]:
diagonal_drift_rate = foff_hz/tsamp_s
diagonal_drift_rate

58.20766091346741